In [ ]:
!pip install scikit-learn==0.23.1
!pip install sklearn_crfsuite

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.8 MB 48.5 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.0.2
    Uninstalling scikit-learn-1.0.2:
      Successfully uninstalled scikit-learn-1.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yellowbrick 1.5 requires scikit-learn>=1.0.0, but you have scikit-learn 0.23.1 which is incompatible.
imbalanced-learn 0.8.1 requires scikit-learn>=0.24, but you have scikit-learn 0.23.1 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 965 kB 15.1 MB/s 


In [ ]:
from google.colab import drive
drive.mount("/gdrive", force_remount=True)

Mounted at /gdrive


**CRFs를 이용해서 자동 띄어쓰기를 수행하는 프로그램**
---
sequence labeling

입력(관측): 한글 음절

출력(레이블): B, I -> BIO 노테이션이라고 함

데이터 형식: 한글 음절 열 \t 레이블 열

예시: 나 는 사 과 가 좋 아 \t B I B I I B I

In [ ]:
import os
import sklearn_crfsuite
from sklearn_crfsuite import metrics


# 파일 경로
file_path = "/gdrive/MyDrive/machine learning/labeling_data.txt"

# "labeling_data.text.txt" 파일을 읽고 lines에 읽은 데이터를 저장
with open(file_path, "r", encoding="utf8") as inFile:
    lines = inFile.readlines()

# 데이터를 음절로 이루어진 문장과 정답 값으로 나누어 저장
datas = []

for line in lines:
  pieces = line.strip().split("\t")
  eumjeol_sequence, label = pieces[0].split(), pieces[1].split()
  datas.append((eumjeol_sequence, label))

number_of_train_datas = int(len(datas)*0.9)

train_datas = datas[:number_of_train_datas]
test_datas = datas[number_of_train_datas:]
          
print("train_datas 개수 : " + str(len(train_datas)))          
print("test_datas 개수 : " + str(len(test_datas)))

train_datas 개수 : 9000
test_datas 개수 : 1000


In [ ]:
# def IS_KOREAN(input):
#    if ord('가') <= ord(input) <= ord('힣'):
#      return True
#    else:
#      return False

In [ ]:
def sent2feature(eumjeol_sequence):
  features = []
  sequence_length = len(eumjeol_sequence)
  for index, eumjeol in enumerate(eumjeol_sequence):
      feature = { "BOS":False, "EOS":False, "WORD":eumjeol, "IS_DIGIT":eumjeol.isdigit(), "IS_PARTICLE":False }

      if(index == 0):
        feature["BOS"] = True
      elif(index == sequence_length-1):
        feature["EOS"] = True
      
      if(index-1 >= 0):
        feature["-1_WORD"] = eumjeol_sequence[index-1]
      if(index-2 >= 0):
        feature["-2_WORD"] = eumjeol_sequence[index-2]
      if(index-3 >= 0):
        feature["-3_WORD"] = eumjeol_sequence[index-3]

      if(index+1 <= sequence_length-1):
        feature["+1_WORD"] = eumjeol_sequence[index+1]
      if(index+2 <= sequence_length-1):
        feature["+2_WORD"] = eumjeol_sequence[index+2]
      if(index-2 >= sequence_length-1):
        feature["+3_WORD"] = eumjeol_sequence[index+3]
      
      # 영어면 B~I까지 음절 띄어쓰기 넣기  /효과가 좋지 않아서 제외
      # if(eumjeol.encode().isalpha()):
      #  feature["IS_ALPHA"] = True

      if(eumjeol == '은' or eumjeol == '는' or eumjeol == '이' or eumjeol == '가' or eumjeol == '을' or eumjeol == '를' or eumjeol == '다' or eumjeol == '며'):
        feature["IS_PARTICLE"] = True

      # if(IS_KOREAN(eumjeol)):
      #   feature["IS_KOREAN"] = True

      features.append(feature)

  return features

In [ ]:
train_x, train_y = [], []
for eumjeol_sequence, label in train_datas:
    train_x.append(sent2feature(eumjeol_sequence))
    train_y.append(label)

test_x, test_y = [], []
for eumjeol_sequence, label in test_datas:
    test_x.append(sent2feature(eumjeol_sequence))
    test_y.append(label)

print(train_x[0:5])
print()
print(train_y[0:5])

[[{'BOS': True, 'EOS': False, 'WORD': '의', 'IS_DIGIT': False, 'IS_PARTICLE': False, '+1_WORD': '료', '+2_WORD': '기'}, {'BOS': False, 'EOS': False, 'WORD': '료', 'IS_DIGIT': False, 'IS_PARTICLE': False, '-1_WORD': '의', '+1_WORD': '기', '+2_WORD': '관'}, {'BOS': False, 'EOS': False, 'WORD': '기', 'IS_DIGIT': False, 'IS_PARTICLE': False, '-1_WORD': '료', '-2_WORD': '의', '+1_WORD': '관', '+2_WORD': '이'}, {'BOS': False, 'EOS': False, 'WORD': '관', 'IS_DIGIT': False, 'IS_PARTICLE': False, '-1_WORD': '기', '-2_WORD': '료', '-3_WORD': '의', '+1_WORD': '이', '+2_WORD': '문'}, {'BOS': False, 'EOS': False, 'WORD': '이', 'IS_DIGIT': False, 'IS_PARTICLE': True, '-1_WORD': '관', '-2_WORD': '기', '-3_WORD': '료', '+1_WORD': '문', '+2_WORD': '을'}, {'BOS': False, 'EOS': False, 'WORD': '문', 'IS_DIGIT': False, 'IS_PARTICLE': False, '-1_WORD': '이', '-2_WORD': '관', '-3_WORD': '기', '+1_WORD': '을', '+2_WORD': '닫'}, {'BOS': False, 'EOS': False, 'WORD': '을', 'IS_DIGIT': False, 'IS_PARTICLE': True, '-1_WORD': '문', '-2_WORD': '이'

In [ ]:
#crf = sklearn_crfsuite.CRF(
#        algorithm='lbfgs',
#        c1=0.1,
#        c2=0.1,
#        max_iterations=100,
#        all_possible_transitions=True
#    )
crf = sklearn_crfsuite.CRF()
crf.fit(train_x, train_y)

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:213: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  FutureWarning)


CRF(keep_tempfiles=None)

**accuracy**

기본 accuracy: 0.9181170653219002

IS_ALPHA 을 넣었을 때: 0.9180974432431371 오히려 정확도가 감소함 -> 제거

IS_PARTICLE 을 넣었을 때: 0.9182347977944784

IS_PARTICLE, IS_KOREAN 을 넣었을 때: 0.9181759315581893 정확도가 감소함 -> 제거

3, -3, IS_PARTICLE 을 넣었을 때: 0.9185683731334497 정확도가 0.0044 오름

이미 약 92%의 accuracy를 가지고 있는 모델이라서 feature을 뭐로 삼든 다 극소치만 오르는 듯...

In [ ]:
def show_predict_result(test_datas, predict):
  for index_1 in range(len(test_datas)):
      eumjeol_sequence, correct_labels = test_datas[index_1]
      predict_labels = predict[index_1]
     
      correct_sentence, predict_sentence = "", ""
      for index_2 in range(len(eumjeol_sequence)):
          if(index_2 == 0):
              correct_sentence += eumjeol_sequence[index_2]
              predict_sentence += eumjeol_sequence[index_2]
              continue

          if(correct_labels[index_2] == "B"):
              correct_sentence += " "
          correct_sentence += eumjeol_sequence[index_2]

          if (predict_labels[index_2] == "B"):
              predict_sentence += " "
          predict_sentence += eumjeol_sequence[index_2]

      print("정답 문장 : " + correct_sentence)
      print("출력 문장 : " + predict_sentence)
      print()

predict = crf.predict(test_x)

print("Accuracy score : " + str(metrics.flat_accuracy_score(test_y, predict)))
print()

print("10개의 데이터에 대한 모델 출력과 실제 정답 비교")
print()

show_predict_result(test_datas[:10], predict[:10])

Accuracy score : 0.9185683731334497

10개의 데이터에 대한 모델 출력과 실제 정답 비교

정답 문장 : 한화그룹(회장 (이름))의 공식 sns 채널 ‘한화데이즈’가 디지털 미디어 시상식 ‘앤 어워드’에서 디지털 ad 부문 그랑프리, 모바일 웹 부문(2개 부문), 디지털 필름 등 4관왕에 올랐다.
출력 문장 : 한화그룹(회장 (이름))의 공식 sns채널 ‘한화데이즈’가 디지털 미디어시상식 ‘앤어워드’에서 디지털ad부문 그랑프리, 모바일 웹부문(2개부문), 디지털 필름등 4관왕에 올랐다.

정답 문장 : '이맹희 별세' . . 고 (이름) 삼성그룹 창업주의 장남이자 (이름) cj그룹 회장의 부친인 이맹희 전 제일비료 회장이 중국에서 별세했다.
출력 문장 : '이 맹희 별세'. .고 (이름) 삼성그룹 창업주의 장남이자 (이름) cj그룹 회장의 부친인이 맹희 전제일 비료 회장이 중국에서 별세했다.

정답 문장 : 제일모직은 17일 오전 임시 주주총회를 열고 삼성물산과의 합병안을 승인했다.
출력 문장 : 제일 모직은 17일 오전 임시 주주 총회를 열고 삼성물산과의 합병안을 승인했다.

정답 문장 : . 보해양조(주)는 17일 동여수노인복지관(관장 정금칠)을 찾아 ‘작은 나눔 큰 사랑 캠페인’에서 모은 성금 200만원으로 구입한 물리치료기기를 전달하고 있다.
출력 문장 : . 보해 양조(주)는 17일동 여수노인복지관(관장 정금칠)을 찾아 ‘작은 나눔 큰 사랑 캠페인’에서 모은 성금 200만원으로 구입한 물리치료 기기를 전달하고 있다.

정답 문장 : 한화큐셀과 한화차이나는 중국청소년발전기금회와 함께 중국 빈곤지역 학교에 태양광 발전설비를 기증하는 캠페인이 두번째 결실을 맺었다고 28일 밝혔다.
출력 문장 : 한화큐셀과 한화차이 나는 중국 청소년 발전기금회와 함께 중국빈곤지역학교에 태양광발전 설비를 기증하는 캠페인이 두번째 결실을 맺었다고 28일 밝혔다.

정답 문장 : 알츠하이머성 치매 증상 완화제 사용량이 비용 기준으로 연평균 두 자릿수씩